In [1]:
# importing modules required
import pandas as pd
from bs4 import BeautifulSoup 
import requests
import time 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import re

### Loading all recipe links

In [9]:
# initiating automated test software
options = Options()
b = webdriver.Chrome(options=options)

In [16]:
# url for sanjeev kapoor website
url = "https://www.sanjeevkapoor.com/AllRecipes/"
b.get(url)

# tested out higher number, there are 2395 times "view more" can be clicked
for i in range(2395):
    time.sleep(0.25)
    # find and click "View More" button
    btn = b.find_element("id", "ctl00_ContentPlaceHolder1_btnmore")
    btn.click()
    
#     print(i)
    
    doc = b.page_source
    soup = BeautifulSoup(b.page_source)

# close testing page
b.close()

In [48]:
# create list of all links on the completely loaded website
recipe_urls = pd.Series([a.get("href") for a in soup.find_all("a")])
recipe_urls = recipe_urls[(recipe_urls.str.count("/Recipe/")>0)].unique()

In [49]:
len(recipe_urls)

13289

In [50]:
recipe_urls

array(['/Recipe/Taazi-Haldi-aur-Gajar-ki-Sabzi.html',
       '/Recipe/Jackfruit-Seed-Sambhar.html',
       '/Recipe/-SABUDANA-VADA.html', ...,
       '../Recipe/Hariyali-Tawa-Machli.html',
       '../Recipe/One-Way-French-Toast.html',
       '../Recipe/Rosemary-Ice-Cream.html'], dtype=object)

In [55]:
# cleaning list of recipe urls
recipe_urls_clean = []
for i in range(len(recipe_urls)):
    # making sure the urls all start with "/Recipe"
    recipe_urls_clean.append(recipe_urls[i][recipe_urls[i].find('/'):])

In [56]:
# initializing data frame
df = pd.DataFrame()

In [57]:
# adding clean recupe_urls as a column to the new data frame
df['recipe_urls'] = recipe_urls_clean
# adding root url to the scraped url, to make url a link
df['recipe_urls'] = "https://www.sanjeevkapoor.com" + df['recipe_urls'].astype('str')

In [58]:
df.head(5)

,recipe_urls
0,https://www.sanjeevkapoor.com/Recipe/Taazi-Hal...
1,https://www.sanjeevkapoor.com/Recipe/Jackfruit...
2,https://www.sanjeevkapoor.com/Recipe/-SABUDANA...
3,https://www.sanjeevkapoor.com/Recipe/PUNJABI-S...
4,https://www.sanjeevkapoor.com/Recipe/PANI-PURI...


In [59]:
# exporting data from with clean urls
df.to_csv('df_url_final.csv')

### Extracting relevant information from recipe urls

In [11]:
# importing url data set
df = pd.read_csv('df_url_final.csv')[['recipe_urls']]

In [13]:
# initializing columns to be filled 
df['recipe_name'] = ""
df['ingredients'] = ""
df['prep_time'] = ""
df['cook_time'] = ""
df['serve'] = ""
df['level_of_cooking'] = ""
df['taste'] = ""

In [14]:
df.head()

,recipe_urls,recipe_name,ingredients,prep_time,cook_time,serve,level_of_cooking,taste
0,https://www.sanjeevkapoor.com/Recipe/Taazi-Hal...,,,,,,,
1,https://www.sanjeevkapoor.com/Recipe/Jackfruit...,,,,,,,
2,https://www.sanjeevkapoor.com/Recipe/-SABUDANA...,,,,,,,
3,https://www.sanjeevkapoor.com/Recipe/PUNJABI-S...,,,,,,,
4,https://www.sanjeevkapoor.com/Recipe/PANI-PURI...,,,,,,,


In [84]:
def extracting_and_adding(cook_time):
    
    '''
    Input: cook time 
    
    Extracts number of servings, level of cooking and taste from characteristics.
    Adds preparation time, cook time, number of servings, level of cooking and taste to data frame for each recipe.
    '''
    
    # extracting serve time, level of cooking and taste from characteristics
    serve = characteristics[2].text.split(': ')[1]
    level_of_cooking = characteristics[3].text.split(': ')[1]
    taste = characteristics[4].text.split(': ')[1]
                
    # adding characteristics to data frame
    df['prep_time'][i] = prep_time 
    df['cook_time'][i] = cook_time 
    df['serve'][i] = serve
    df['level_of_cooking'][i] = level_of_cooking
    df['taste'][i] = taste

In [85]:
def if_unit_exists(unit, time_scrubbed):
    
    '''
    Input: unit and time of cook time or preparation time
    
    Checks if unit is in minutes, if it is in minutes, then leave as is.
    If unit is in hours, convert to minutes. 
    Returns cook or preparation time in minutes with format xx-xx.
    '''
    
    # if unit is in minutes, then don't convert 
    if unit == 'minutes':
        time = time_scrubbed.split(' ')[0]
        
    # else, if unit is in hours, convert to minutes
    elif ((unit == 'hour') or (unit == 'hours')):

        # time is in format "xx-xx" 
        # scrub first and second half of time and convert to minutes
        time_range = time_scrubbed.split(' ')[0]
        time_min = float(time_range.split('-')[0])*60
        time_max = float(time_range.split('-')[1])*60
        # combine time in minutes to "xx-xx" format
        time = str(time_min)+'-'+str(time_max)
    
    return time

In [86]:
def cook_time_check_and_others_extraction():
    
    '''
    Checks if cook time unit exists. 
    If there is a cook time with appropriate units, then applies if_unit_exists().
    If there is no cook time with approproate units, then cook time is empty.
    Adds all extracted data to data frame.
    '''

    # check if cook time exists
    if (exp1_cook or exp2_cook or exp3_cook or exp4_cook):
        
        cook_time = if_unit_exists(cook_time_unit, cook_time_scrubbed)
        
        # extracting serve time, level of cooking and taste from characteristics
        extracting_and_adding(cook_time)
        
    # if cook time unit does not exist, execute this
    else:
        cook_time = ""
        
        # extracting serve time, level of cooking and taste from characteristics
        extracting_and_adding(cook_time)

In [89]:
# scrubbing relevant information about each recipe in data set
for i in range(len(df)):

    # selecting a recipe url
    url = df['recipe_urls'][i]
    # getting content from recipe url 
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')
    
    try:
        # extracting recipe name and adding to data set
        recipe_name = soup.find('h1').text.strip().split(' Recipe Card')[0].title()
        df['recipe_name'][i] = recipe_name
    # if recipe name does not exist, continue to next loop
    except:
        continue    
    else:
        # if recipe name is "Oops!", it does not exist, continue to next loop
        if (recipe_name == "Oops!"):
            continue
        
        # if recipe name does exist, execute this    
        else:
            # scrubbing ingredients into a list
            ingredients = []
            for li in soup.select('.list-unstyled li'):
                ingredient = ' '.join(li.text.split())
                ingredients.append(ingredient)
                
            # adding ingredient list to data frame
            df['ingredients'][i] = ingredients

            # recipe characteristics are: prep time, cook time, serving size, level of cooking, and taste
            # scrubbing recipe characteristics
            characteristics = soup.select('div.col-md-12.col-xs-12.preptime.prepretime.col-sm-12 > div > p')
            
            # prep time and cook time extracted from charactersitics
            prep_time_scrubbed = characteristics[0].text.split(': ')[1]
            cook_time_scrubbed = characteristics[1].text.split(': ')[1]

            # check to see if there is prep time unit
            try:
                prep_time_unit = prep_time_scrubbed.split(' ')[1]
                exp1_prep = prep_time_unit == 'minute'
                exp2_prep = prep_time_unit == 'minutes'
                exp3_prep = prep_time_unit == 'hour'
                exp4_prep = prep_time_unit == 'hours'

                cook_time_unit = cook_time_scrubbed.split(' ')[1]
                exp1_cook = cook_time_unit == 'minute'
                exp2_cook = cook_time_unit == 'minutes'
                exp3_cook = cook_time_unit == 'hour'
                exp4_cook = cook_time_unit == 'hours'

                # if prep time unit does not exist, then execute this
                if not (exp1_prep or exp2_prep or exp3_prep or exp4_prep):
                    # extracting prep time without unit, and converting to minutes 
                    prep_time = ""

                    # extract cook time and add all characteristics to data set 
                    cook_time_check_and_others_extraction()

                # if prep time unit exists, then execute this
                else:        

                    # if prep time unit exists
                    prep_time = if_unit_exists(prep_time_unit, prep_time_scrubbed)

                    # extract cook time and add all characteristics to data set
                    cook_time_check_and_others_extraction()
                    
            except:
                prep_time = float(prep_time_scrubbed.split('-')[0])*60
                
                # extract cook time and add all characteristics to data set
                cook_time_check_and_others_extraction()
                

In [88]:
df.head()

,recipe_urls,recipe_name,ingredients,prep_time,cook_time,serve,level_of_cooking,taste
0,https://www.sanjeevkapoor.com/Recipe/Taazi-Hal...,Taazi Haldi Aur Gajar Ki Sabzi,"[Fresh turmeric (taazi haldi) , chopped 1 1/2 ...",11-15,21-25,4,Easy,Spicy
1,https://www.sanjeevkapoor.com/Recipe/Jackfruit...,Jackfruit Seed Sambhar,"[Ripe jackfruit kernels 20-25, Salt to taste, ...",41-50,41-50,4,Moderate,Spicy
2,https://www.sanjeevkapoor.com/Recipe/-SABUDANA...,Sabudana Vada,"[Sago (sabudana) ,soaked 1 1/2 cups, Chutney, ...",360.0-420.0,11-15,4,Moderate,Mild
3,https://www.sanjeevkapoor.com/Recipe/PUNJABI-S...,Punjabi Samosa,"[Refined flour (maida) 1 1/2 cups, Potatoes , ...",31-40,41-50,4,Difficult,Tangy
4,https://www.sanjeevkapoor.com/Recipe/PANI-PURI...,Pani Puri,"[Puris as required, Potatoes , boiled and peel...",31-40,0-5,4,Moderate,Tangy


In [65]:
# exporting scrubbed data set
# df.to_csv('df_scrubbed_final.csv')
# DO NOT DELETE DO NOT RUN
# df.to_csv('df_scrubbed.csv')